<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Gabriela Mora
- Nombre de alumno 2: Raúl Silva


### **Link de repositorio de GitHub:** `https://github.com/RaulSilvaA/MDS7202-mora-silva`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 74.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=069d6fe245a7d18e83638511b9b916780bfdb35752a764d202daa26280811d70
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow.

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados.
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/lab_progra/'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [3]:
df_comics = pd.read_csv(path+'df_comics.csv')
df_comics_no_label = pd.read_csv(path+'comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
df_comics.shape

(1285, 82)

In [7]:
df_comics_no_label.head(5)

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


In [8]:
df_comics.columns

Index(['Unnamed: 0', 'name', 'real_name', 'full_name', 'overall_score',
       'history_text', 'powers_text', 'intelligence_score', 'strength_score',
       'speed_score', 'durability_score', 'power_score', 'combat_score',
       'superpowers', 'alter_egos', 'aliases', 'place_of_birth',
       'first_appearance', 'creator', 'alignment', 'occupation', 'base',
       'teams', 'relatives', 'gender', 'type_race', 'height', 'weight',
       'eye_color', 'hair_color', 'skin_color', 'img', 'has_electrokinesis',
       'has_energy_constructs', 'has_mind_control_resistance',
       'has_matter_manipulation', 'has_telepathy_resistance',
       'has_mind_control', 'has_enhanced_hearing', 'has_dimensional_travel',
       'has_element_control', 'has_size_changing', 'has_fire_resistance',
       'has_fire_control', 'has_dexterity', 'has_reality_warping',
       'has_illusions', 'has_energy_beams', 'has_peak_human_condition',
       'has_shapeshifting', 'has_heat_resistance', 'has_jump',
       'has_

In [9]:
df_comics['alignment']

0       Good
1       Good
2       Good
3       Good
4       Good
        ... 
1362    Good
1363    Good
1364     Bad
1365     Bad
1366     Bad
Name: alignment, Length: 1285, dtype: object

In [10]:
df_comics_no_label['alignment']

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
      ..
79   NaN
80   NaN
81   NaN
82   NaN
83   NaN
Name: alignment, Length: 84, dtype: float64

Efectivamente, la columna a predecir es `alignment`

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset.

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen.

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?.

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [4]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']

nltk.download('punkt')
docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [5]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [7]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [6]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados.

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [7]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [X] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [X] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [8]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
text = 'history_text'

In [9]:
#Scaller
minmax = MinMaxScaler()
#BagOfWords
bow = CountVectorizer(tokenizer = StemmerTokenizer(),
                      ngram_range=(1,2)
                      )

In [12]:
#Prepro
preprocesador = ColumnTransformer(
    transformers=[
        ('BoW', bow, text),
        ('MinMax', minmax, atributos_de_interes),
    ],
    remainder='drop',
    sparse_threshold=0
    )

In [13]:
pipeline = Pipeline([
    ('prepro', preprocesador),
])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


# Nueva sección
Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`.

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

In [ ]:
df_comics.head(5)

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Respuesta:**

In [14]:
clf = Pipeline([
    ('prepr', preprocesador),
    ('seleccion', SelectPercentile(score_func=f_classif, percentile=90)),
    ('clasificador', MultinomialNB())
])

X = df_comics.drop(['alignment', 'name',	'real_name',	'full_name', 	'powers_text'], axis=1)
y = df_comics['alignment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=190423)

In [15]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
#reportamos resultados
report = classification_report(y_test, y_pred)

In [17]:
#Dummy
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_test)
dummy_report = classification_report(y_test, y_pred_dummy)

In [18]:
print(report)

              precision    recall  f1-score   support

         Bad       0.80      0.10      0.18        77
        Good       0.63      0.99      0.77       156
     Neutral       1.00      0.04      0.08        24

    accuracy                           0.64       257
   macro avg       0.81      0.38      0.35       257
weighted avg       0.72      0.64      0.53       257



In [19]:
print(dummy_report)

              precision    recall  f1-score   support

         Bad       0.30      0.42      0.35        77
        Good       0.56      0.47      0.51       156
     Neutral       0.05      0.04      0.05        24

    accuracy                           0.41       257
   macro avg       0.30      0.31      0.30       257
weighted avg       0.43      0.41      0.42       257




¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?

No, porque los resultados no son suficientemente alentadores para asegurarnos que las predicción serán correctas en datos no etiquetados con anterioridad

¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

Para la clase malo es mejor, pero para el resto no. Esto nos puede ayudar a concluir que el BoW si entrega información relevante para etiquetar personajes.


## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [20]:
### Configuraciones de BoW ###
bow1 = CountVectorizer(tokenizer = StemmerTokenizer(),
                      ngram_range=(1,1),
                      max_df=2
                      )

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import svm

#### Código aquí ####

### Modificaciones en la representación del BoW + otras características ###
preprocesador1 = ColumnTransformer(
    transformers=[
        ('BoW', bow1, text),
        ('MinMax', minmax, atributos_de_interes),
    ],
    remainder='drop',
    sparse_threshold=0
    )


In [22]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [23]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

In [24]:
from sklearn.metrics import f1_score


def train_and_evaluate(
    pipe, print_=True, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test
):

    # notar que los datasets son parámetros por defecto.

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    if print_:
        print("Matriz de confusión: \n")
        print(confusion_matrix(y_test, y_pred, labels=pipe.classes_))
        print("\nReporte de Clasificación: \n")
        print(
            classification_report(y_test, y_pred, target_names=pipe.classes_),
        )

    return f1_score(y_test, y_pred, average="macro")

## Primer clasificador

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline

selection_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocesador1),
        ("selection", SelectPercentile(f_classif, percentile=20)),
        ("model", RandomForestClassifier()),
    ]
)

In [39]:
param_grid = [
    {
        "selection__percentile": range(40, 60, 80),
        "model__criterion": ["gini", "entropy"],
        "model__bootstrap": [True, False],
        "preprocessing__BoW__ngram_range": [(1, 1), (1, 2), (1, 3)],
        "preprocessing__BoW__max_df": [0,1,2],
        "preprocessing__BoW__min_df": [0,1,2]
    }
]

hgs = HalvingGridSearchCV(selection_pipeline, param_grid, n_jobs=-1, scoring='f1_macro', verbose=10)

In [40]:
train_and_evaluate(hgs)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 108
n_resources: 30
Fitting 5 folds for each of 108 candidates, totalling 540 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-p

----------
iter: 1
n_candidates: 36
n_resources: 90
Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.23396825 0.24666667 0.24666667
 0.23396825 0.24666667 0.24666667        nan        nan        nan
 0.21174603 0.21174603 0.24666667 0.21174603 0.21174603 0.24666667
 0.30960317 0.25626984 0.25309524        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.24666667 0.24666667 0.24666667 0.24666667 0.24666667 0.24666667
        nan        nan        nan 0.22285714 0.24666667 0.24666667
 0.21619048 0.21174603 0.24666667 0.28960317 0.3097619  0.27309524
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.21015873 0.24666667 0.24666667
 0.23396825 0.24666667 0.24666667        nan        nan        nan
 0.26952381 0.21174603 0.22285714 0.19238095 0.22285714 

----------
iter: 2
n_candidates: 12
n_resources: 270
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.23396825 0.24666667 0.24666667
 0.23396825 0.24666667 0.24666667        nan        nan        nan
 0.21174603 0.21174603 0.24666667 0.21174603 0.21174603 0.24666667
 0.30960317 0.25626984 0.25309524        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.24666667 0.24666667 0.24666667 0.24666667 0.24666667 0.24666667
        nan        nan        nan 0.22285714 0.24666667 0.24666667
 0.21619048 0.21174603 0.24666667 0.28960317 0.3097619  0.27309524
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.21015873 0.24666667 0.24666667
 0.23396825 0.24666667 0.24666667        nan        nan        nan
 0.26952381 0.21174603 0.22285714 0.19238095 0.22285714 

----------
iter: 3
n_candidates: 4
n_resources: 810
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.23396825 0.24666667 0.24666667
 0.23396825 0.24666667 0.24666667        nan        nan        nan
 0.21174603 0.21174603 0.24666667 0.21174603 0.21174603 0.24666667
 0.30960317 0.25626984 0.25309524        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.24666667 0.24666667 0.24666667 0.24666667 0.24666667 0.24666667
        nan        nan        nan 0.22285714 0.24666667 0.24666667
 0.21619048 0.21174603 0.24666667 0.28960317 0.3097619  0.27309524
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.21015873 0.24666667 0.24666667
 0.23396825 0.24666667 0.24666667        nan        nan        nan
 0.26952381 0.21174603 0.22285714 0.19238095 0.22285714 

Matriz de confusión: 

[[ 20  55   2]
 [ 33 119   4]
 [  8  15   1]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

         Bad       0.33      0.26      0.29        77
        Good       0.63      0.76      0.69       156
     Neutral       0.14      0.04      0.06        24

    accuracy                           0.54       257
   macro avg       0.37      0.35      0.35       257
weighted avg       0.49      0.54      0.51       257



0.34807542465326474

In [41]:
hgs.best_params_

{'model__bootstrap': False,
 'model__criterion': 'entropy',
 'preprocessing__BoW__max_df': 2,
 'preprocessing__BoW__min_df': 2,
 'preprocessing__BoW__ngram_range': (1, 1),
 'selection__percentile': 40}

## Segundo clasificador

In [42]:
selection_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocesador1),
        ("selection", SelectPercentile(f_classif, percentile=20)),
        ("model", LinearSVC(random_state=0, tol=1e-5)),
    ]
)

In [47]:
param_grid = [
    {
        "selection__percentile": range(40, 60, 80),
        "model__C": [0.1,1, 10, 100],
        "preprocessing__BoW__ngram_range": [(1, 1), (1, 2), (1, 3)],
        "preprocessing__BoW__max_df": [0,1,2],
        "preprocessing__BoW__min_df": [0,1,2]
    }
]

hgs1 = HalvingGridSearchCV(selection_pipeline, param_grid, n_jobs=-1, scoring='f1_macro', verbose=10)

In [48]:
train_and_evaluate(hgs1)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 108
n_resources: 30
Fitting 5 folds for each of 108 candidates, totalling 540 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-p

----------
iter: 1
n_candidates: 36
n_resources: 90
Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26087302 0.26087302 0.26087302
 0.26087302 0.26087302 0.26087302        nan        nan        nan
 0.26642857 0.26087302 0.26087302 0.26642857 0.26087302 0.26087302
 0.34087302 0.28902116 0.28902116        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.26087302 0.26087302 0.26087302 0.26087302 0.26087302 0.26087302
        nan        nan        nan 0.26642857 0.26087302 0.26087302
 0.26642857 0.26087302 0.26087302 0.31230159 0.24140212 0.24140212
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26087302 0.26087302 0.26087302
 0.26087302 0.26087302 0.26087302        nan        nan        nan
 0.26642857 0.26087302 0.26087302 0.26642857 0.26087302 

----------
iter: 2
n_candidates: 12
n_resources: 270
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26087302 0.26087302 0.26087302
 0.26087302 0.26087302 0.26087302        nan        nan        nan
 0.26642857 0.26087302 0.26087302 0.26642857 0.26087302 0.26087302
 0.34087302 0.28902116 0.28902116        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.26087302 0.26087302 0.26087302 0.26087302 0.26087302 0.26087302
        nan        nan        nan 0.26642857 0.26087302 0.26087302
 0.26642857 0.26087302 0.26087302 0.31230159 0.24140212 0.24140212
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26087302 0.26087302 0.26087302
 0.26087302 0.26087302 0.26087302        nan        nan        nan
 0.26642857 0.26087302 0.26087302 0.26642857 0.26087302 

----------
iter: 3
n_candidates: 4
n_resources: 810
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26087302 0.26087302 0.26087302
 0.26087302 0.26087302 0.26087302        nan        nan        nan
 0.26642857 0.26087302 0.26087302 0.26642857 0.26087302 0.26087302
 0.34087302 0.28902116 0.28902116        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.26087302 0.26087302 0.26087302 0.26087302 0.26087302 0.26087302
        nan        nan        nan 0.26642857 0.26087302 0.26087302
 0.26642857 0.26087302 0.26087302 0.31230159 0.24140212 0.24140212
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26087302 0.26087302 0.26087302
 0.26087302 0.26087302 0.26087302        nan        nan        nan
 0.26642857 0.26087302 0.26087302 0.26642857 0.26087302 

Matriz de confusión: 

[[ 11  63   3]
 [ 21 128   7]
 [  3  19   2]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

         Bad       0.31      0.14      0.20        77
        Good       0.61      0.82      0.70       156
     Neutral       0.17      0.08      0.11        24

    accuracy                           0.55       257
   macro avg       0.36      0.35      0.34       257
weighted avg       0.48      0.55      0.49       257



0.3356644114840836

In [49]:
hgs1.best_params_

{'model__C': 10,
 'preprocessing__BoW__max_df': 2,
 'preprocessing__BoW__min_df': 2,
 'preprocessing__BoW__ngram_range': (1, 1),
 'selection__percentile': 40}

## Tercer clasificador

In [25]:
selection_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocesador1),
        ("selection", SelectPercentile(f_classif, percentile=20)),
        ("model", svm.SVC()),
    ]
)

In [26]:
param_grid = [
    {
        "selection__percentile": range(40, 60, 80),
        "model__C": [0.1,1, 10, 100],
        "preprocessing__BoW__ngram_range": [(1, 1), (1, 2), (1, 3)],
        "preprocessing__BoW__max_df": [0,1,2],
        "preprocessing__BoW__min_df": [0,1,2]
    }
]


hgs2 = HalvingGridSearchCV(selection_pipeline, param_grid, n_jobs=-1, scoring='f1_macro', verbose=10)

In [27]:
train_and_evaluate(hgs2)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 108
n_resources: 30
Fitting 5 folds for each of 108 candidates, totalling 540 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-p

----------
iter: 1
n_candidates: 36
n_resources: 90
Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26904762 0.26904762 0.26904762
 0.26904762 0.26904762 0.26904762        nan        nan        nan
 0.26904762 0.26904762 0.26904762 0.26904762 0.26904762 0.26904762
 0.26904762 0.26904762 0.26904762        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.31428571 0.27619048 0.26904762 0.31428571 0.27619048 0.26904762
        nan        nan        nan 0.27619048 0.26904762 0.26904762
 0.27619048 0.26904762 0.26904762 0.25571429 0.26904762 0.26904762
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.28428571 0.27619048 0.27619048
 0.28428571 0.27619048 0.27619048        nan        nan        nan
 0.32238095 0.27619048 0.33571429 0.32238095 0.27619048 

----------
iter: 2
n_candidates: 12
n_resources: 270
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26904762 0.26904762 0.26904762
 0.26904762 0.26904762 0.26904762        nan        nan        nan
 0.26904762 0.26904762 0.26904762 0.26904762 0.26904762 0.26904762
 0.26904762 0.26904762 0.26904762        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.31428571 0.27619048 0.26904762 0.31428571 0.27619048 0.26904762
        nan        nan        nan 0.27619048 0.26904762 0.26904762
 0.27619048 0.26904762 0.26904762 0.25571429 0.26904762 0.26904762
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.28428571 0.27619048 0.27619048
 0.28428571 0.27619048 0.27619048        nan        nan        nan
 0.32238095 0.27619048 0.33571429 0.32238095 0.27619048 

----------
iter: 3
n_candidates: 4
n_resources: 810
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.26904762 0.26904762 0.26904762
 0.26904762 0.26904762 0.26904762        nan        nan        nan
 0.26904762 0.26904762 0.26904762 0.26904762 0.26904762 0.26904762
 0.26904762 0.26904762 0.26904762        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.31428571 0.27619048 0.26904762 0.31428571 0.27619048 0.26904762
        nan        nan        nan 0.27619048 0.26904762 0.26904762
 0.27619048 0.26904762 0.26904762 0.25571429 0.26904762 0.26904762
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.28428571 0.27619048 0.27619048
 0.28428571 0.27619048 0.27619048        nan        nan        nan
 0.32238095 0.27619048 0.33571429 0.32238095 0.27619048 

Matriz de confusión: 

[[ 14  61   2]
 [ 22 129   5]
 [  0  21   3]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

         Bad       0.39      0.18      0.25        77
        Good       0.61      0.83      0.70       156
     Neutral       0.30      0.12      0.18        24

    accuracy                           0.57       257
   macro avg       0.43      0.38      0.38       257
weighted avg       0.52      0.57      0.52       257



0.37575182468637425

In [28]:
hgs2.best_params_


{'model__C': 100,
 'preprocessing__BoW__max_df': 1,
 'preprocessing__BoW__min_df': 0,
 'preprocessing__BoW__ngram_range': (1, 1),
 'selection__percentile': 40}

Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

Se elige el clasificador de svm.SVC con el único parámetro variado de C igual a 100 y con valores de CountVectorizer con 1-ngram, con df_max=1 y df_min=0. Y por último en cuando al Selectionpercentile, el elegido es el de 40.

La razón para elegir este clasificador es porque es el que da un mejor valor de F1-Score en comparación a los otros modelos. Se elige la métrica F1-Score debido a que nos interesan todos los tipos de las etiquetas, y esa métrica calcula un ponderado de las clases positivas que realmente nos interesan.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [20]:
from sklearn import svm

In [14]:
params = {'model__C': 100,
 'preprocessing__BoW__max_df': 1,
 'preprocessing__BoW__min_df': 0,
 'preprocessing__BoW__ngram_range': (1, 1),
 'selection__percentile': 40}

In [17]:
### Configuraciones de BoW ###
bow2 = CountVectorizer(tokenizer = StemmerTokenizer(),
                      ngram_range= params['preprocessing__BoW__ngram_range'],
                      max_df=params['preprocessing__BoW__max_df'],
                      min_df=params['preprocessing__BoW__min_df']
                      )

In [18]:
preprocesador1 = ColumnTransformer(
    transformers=[
        ('BoW', bow2, text),
        ('MinMax', minmax, atributos_de_interes),
    ],
    remainder='drop',
    sparse_threshold=0
    )

In [21]:
#### Código aquí ####
clasificador = Pipeline(
    steps=[
        ("preprocessing", preprocesador1),
        ("selection", SelectPercentile(f_classif, percentile=params['selection__percentile'])),
        ("model", svm.SVC(C=params['model__C'])),
    ]
)

In [22]:
X = df_comics.drop(['alignment', 'name',	'real_name',	'full_name', 	'powers_text'], axis=1)
y = df_comics['alignment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=190423)

In [23]:
clasificador.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('BoW',
                                                  CountVectorizer(max_df=1,
                                                                  min_df=0,
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f4047e3f670>),
                                                  'history_text'),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score'])])),
                ('selection', SelectPercentile(percentile=40)),
                ('model', SVC(C=100))])

In [25]:
nombre_predic=['Vergil', 'Gorilla Girl', 'Batcow']

In [28]:
X_prediccion=df_comics_no_label[df_comics_no_label['name'].isin(nombre_predic)]

In [31]:
X_prediccion=X_prediccion.iloc[:3]

In [34]:
X_prediccion

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [33]:
clasificador.predict(X_prediccion)

array(['Bad', 'Good', 'Neutral'], dtype=object)

# El Batcow es Bad La Gorilla Girl es Good y Vergil es Neutral

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>